![Game Data Exploration](https://sfc-gh-jholt.github.io/jgh-images/banner.jpeg)


In [ ]:
import modin.pandas as pd
import snowflake.snowpark.modin.plugin
from snowflake.snowpark.context import get_active_session
import streamlit as st
from snowflake.snowpark import Session
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from PIL import Image

warnings.filterwarnings('ignore', category=UserWarning, module="snowpark.modin.plugin.utils")

In [ ]:
# This cell imports necessary libraries and sets up the environment
from snowflake.snowpark.context import get_active_session
import streamlit as st
from snowflake.snowpark import Session
import numpy as np
import seaborn as sns
import warnings
from PIL import Image


import modin.pandas as pd
import snowflake.snowpark.modin.plugin
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore', category=UserWarning, module="snowpark.modin.plugin.utils")

In [ ]:
session = get_active_session()
df = pd.read_snowflake("gamestats")

In [ ]:
# Count unique values in each column
unique_counts = df.nunique()

# Display the unique counts
print("Unique counts in each column:")
print(unique_counts)

In [ ]:
st.dataframe(df.head())

In [ ]:
df.columns


In [ ]:
df = df.drop(['NUMBER'], axis=1)  # Adjust based on actual column name


In [ ]:
st.dataframe(df.head())

In [ ]:
print('shape of the dataset=', df.shape)

print(' \nThe null count of each column of the dataset are as follows:')
df.isnull().sum()

In [ ]:
# To view the null row from the dataset:

df[df['WINPLACEPERC'].isnull() == True]

In [ ]:
df = df.dropna(subset=['WINPLACEPERC'])  # Drop rows with NaN values in 'WINPLACEPERC'

In [ ]:
# Function to identify numeric features
def numeric_features(dataset):
    numeric_col = dataset.select_dtypes(include=np.number).columns.tolist()
    return dataset[numeric_col].head()

# Function to identify categorical features
def categorical_features(dataset):
    categorical_col = dataset.select_dtypes(exclude=np.number).columns.tolist()
    return dataset[categorical_col].head()

# Function to check the datatypes of all the columns
def check_datatypes(dataset):
    return dataset.dtypes

# Load your DataFrame (assuming 'df' is already loaded from Snowflake or other source)
# Displaying in Streamlit:

# Title
st.title("Dataframe Overview")

# Display Numerical Features
numeric_columns = numeric_features(df)
st.subheader("Numerical Features")
st.dataframe(numeric_columns)  # Streamlit dataframe for better display

# Separator
st.write("====" * 20)

# Display Categorical Features
categorical_columns = categorical_features(df)
st.subheader("Categorical Features")
st.dataframe(categorical_columns)

# Separator
st.write("====" * 20)



In [ ]:

# Function to detect outliers in every feature
def detect_outliers(df):
    cols = list(df)
    outliers_list = []  # Use a list to store the rows first
    
    for column in cols:
        # Check if the column is numeric
        if column in df.select_dtypes(include=np.number).columns:
            q1 = df[column].quantile(0.25)  # First quartile
            q3 = df[column].quantile(0.75)  # Third quartile
            iqr = q3 - q1  # Interquartile range
            fence_low = q1 - (1.5 * iqr)
            fence_high = q3 + (1.5 * iqr)
            
            # Count the number of outliers
            num_outliers = df.loc[(df[column] < fence_low) | (df[column] > fence_high)].shape[0]
            
            # Append the result as a dictionary to the list
            outliers_list.append({'Feature': column, 'Number of Outliers': num_outliers})
    
    # Convert the list of dictionaries into a DataFrame
    outliers_df = pd.DataFrame(outliers_list)
    
    return outliers_df

# Assuming 'df' is already loaded and contains your data
st.title("Outliers Detection")

# Detect outliers
outliers_df = detect_outliers(df)

# Display outliers in a Streamlit DataFrame
st.subheader("Outliers per Feature")
st.dataframe(outliers_df)


| ![Image 1](https://sfc-gh-jholt.github.io/jgh-images/gif1.gif) | 




In [ ]:
# Assuming df['KILLS'] contains the data

# Summary statistics for the number of KILLS
kills_stats = {
    'Statistic': ['Average Kills', '50th Percentile (Median)', '75th Percentile', '99th Percentile', 'Maximum Kills'],
    'Value': [
        '{:.4f}'.format(df['KILLS'].mean()),
        df['KILLS'].quantile(0.50),
        df['KILLS'].quantile(0.75),
        df['KILLS'].quantile(0.99),
        df['KILLS'].max()
    ]
}

# Convert to a DataFrame
kills_stats_df = pd.DataFrame(kills_stats)

# Display in Streamlit
st.title("Summary Statistics for KILLS")
st.dataframe(kills_stats_df)

In [ ]:
from snowflake.snowpark.functions import col, when, approx_percentile



# Assuming session is already established
# Step 1: Calculate the 99th percentile of the 'KILLS' column
percentile_99_value = session.table("gamestats").select(approx_percentile(col('KILLS'), 0.99)).collect()[0][0]

# Step 2: Retrieve the original Snowpark DataFrame
snowpark_df = session.table("gamestats")

# Step 3: Create a new column 'KILLS_CATEGORIZED' to categorize kills greater than the 99th percentile
df_with_kills_categorized = snowpark_df.with_column(
    'KILLS_CATEGORIZED',
    when(col('KILLS') > percentile_99_value, '8+').otherwise(col('KILLS').cast('string'))
)

# Step 4: Convert the Snowpark DataFrame to a Pandas DataFrame for plotting
pandas_df = df_with_kills_categorized.to_pandas()

# Step 5: Plot the data using Seaborn
plt.figure(figsize=(20, 15))
sns.countplot(x=pandas_df['KILLS_CATEGORIZED'].astype(str).sort_values())  # Convert to string for plotting
plt.title('Kill Count', fontsize=15)
plt.xlabel('KILLS', fontsize=15)
plt.ylabel('Count', fontsize=13)

# Step 6: Display the plot in Streamlit
st.pyplot(plt)

# Step 7: Clear the figure to avoid duplication
plt.clf()
plt.close()

In [ ]:

average_kills = df['KILLSTREAKS'].mean()
median_kills = df['KILLSTREAKS'].quantile(0.50)
kills_75_percentile = df['KILLSTREAKS'].quantile(0.75)
kills_99_percentile = df['KILLSTREAKS'].quantile(0.99)
max_kills = df['KILLSTREAKS'].max()

# Step 1: Display the statistics using Streamlit's st.write()
st.write(f'The average person kills {average_kills:.4f} players in a short time.')
st.write(f'50% of people have {median_kills} kills or less in a short time.')
st.write(f'75% of people have {kills_75_percentile} kills or less in a short time.')
st.write(f'99% of people have {kills_99_percentile} kills or less in a short time.')
st.write(f'While the most kills in a row recorded in the data is {max_kills}.')

In [ ]:

# Assuming df is already loaded


# Step 1: Calculate the 99th percentile for 'KILLSTREAKS'
percentile_99_value = session.table("gamestats").select(approx_percentile(col('KILLSTREAKS'), 0.99)).collect()[0][0]

# Step 2: Retrieve the original Snowpark DataFrame
snowpark_df = session.table("gamestats")

# Step 3: Use SQL-style transformations to categorize 'KILLSTREAKS' above the 99th percentile
df_with_killstreaks_categorized = snowpark_df.with_column(
    'KILLSTREAKS_CATEGORIZED',
    when(col('KILLSTREAKS') > percentile_99_value, '4+').otherwise(col('KILLSTREAKS').cast('string'))
)

# Step 4: Convert the Snowpark DataFrame to Pandas for visualization
pandas_df = df_with_killstreaks_categorized.to_pandas()

# Step 5: Plot the data using Seaborn
plt.figure(figsize=(20, 15))
sns.countplot(x=pandas_df['KILLSTREAKS_CATEGORIZED'].sort_values())
plt.title('Kill Count', fontsize=15)
plt.xlabel('KILLSTREAKS', fontsize=15)
plt.ylabel('Count', fontsize=13)

# Step 6: Display the plot in Streamlit
st.pyplot(plt)

In [ ]:

# Assuming df is a Pandas DataFrame
# Get the value counts of the 'MATCHTYPE' column
matchtype_counts = df['MATCHTYPE'].value_counts().reset_index()

# Rename the columns for better readability
matchtype_counts.columns = ['Match Type', 'Count']

# Step 1: Display the table in Streamlit using st.write or st.dataframe
st.write("### Match Type Counts")
st.dataframe(matchtype_counts)

In [ ]:

# Step 1: Set the plot size
plt.figure(figsize=(20, 15))

# Step 2: Create a countplot for 'MATCHTYPE' with logarithmic scaling on the y-axis
sns.countplot(x=df['MATCHTYPE'], hue=df['MATCHTYPE'], palette="Set2", legend=False)

# Step 3: Customize the plot
plt.title('Match Type with Log Scale', fontsize=15)
plt.xlabel('Match Type', fontsize=15)
plt.ylabel('Count (log scale)', fontsize=13)

# Step 4: Set the y-axis to logarithmic scale
plt.yscale('log')

# Step 5: Display the plot in Streamlit
st.pyplot(plt)


In [ ]:
import pandas as pan


# Step 1: Load the data from Snowpark and convert it to a Pandas DataFrame
data = session.table("gamestats").to_pandas()

# Step 2: Convert all column names to uppercase for consistency
data.columns = [col.upper() for col in data.columns]

# Step 3: Keep only those players that didn't kill anyone
data = data[data['KILLS'] == 0]

# Step 4: Ensure 'DAMAGEDEALT' is numeric and handle NaNs or non-numeric values
data['DAMAGEDEALT'] = pan.to_numeric(data['DAMAGEDEALT'], errors='coerce').fillna(0)

# Step 5: Apply logarithmic transformation to 'DAMAGEDEALT' to reduce skew
data['LOG_DAMAGEDEALT'] = np.log1p(data['DAMAGEDEALT'])  # log1p handles log(0) gracefully

# Step 6: Plot the distribution of log-transformed 'DAMAGEDEALT' using histplot
plt.figure(figsize=(15, 10))
plt.title('Log-Scaled Damage Dealt by 0 Killers', fontsize=15)

# Use histplot to plot the distribution
sns.histplot(data['LOG_DAMAGEDEALT'], kde=True, bins=30)
plt.xlabel('Log Damage Dealt', fontsize=15)
plt.ylabel('Density', fontsize=13)

# Step 7: Display the plot in Streamlit
st.pyplot(plt)

In [ ]:

# Step 1: Ensure 'WINPLACEPERC' is numeric and handle any missing values
data['WINPLACEPERC'] = pan.to_numeric(data['WINPLACEPERC'], errors='coerce')  # Convert to numeric, invalid entries become NaN


# Step 2: Keep only the players that won the match (WINPLACEPERC == 1)
data = data[data['WINPLACEPERC'] == 1]

# Step 3: Set up the plot
plt.figure(figsize=(15, 10))
plt.title('Match Duration for Winners', fontsize=15)

# Step 4: Create the histogram plot for 'MATCHDURATION'
sns.histplot(data['MATCHDURATION'], kde=False)

# Step 5: Label the axes
plt.xlabel('Match Duration', fontsize=15)
plt.ylabel('Density', fontsize=13)

# Step 6: Display the plot in Streamlit
st.pyplot(plt)


# Step 7: Clear the figure to avoid duplication
plt.clf()
plt.close()

In [ ]:
# Step 1: Load the data from Snowflake into a Pandas DataFrame
df = session.table("gamestats").to_pandas()

# Step 2: Set up the plot size and create a jointplot
plt.figure(figsize=(15, 10))
sns.jointplot(x='WINPLACEPERC', y='KILLSTREAKS', data=df, color='b')

# Step 3: Add axis labels
plt.xlabel('Win Place Percentage', fontsize=15)
plt.ylabel('Kill Streaks', fontsize=13)

# Step 4: Display the plot in Streamlit
st.pyplot(plt)

In [ ]:

# Step 1: Set up the jointplot
joint_plot = sns.jointplot(x='WINPLACEPERC', y='DAMAGEDEALT', data=df, kind='scatter', color='b')

# Step 2: Customize plot with labels
joint_plot.set_axis_labels('Win Place Percentage', 'Damage Dealt', fontsize=15)

# Step 3: Add a title using suptitle (since jointplot doesn't work well with plt.title)
plt.suptitle('Win Place vs Damage Dealt', y=1.03, fontsize=16)

# Step 4: Display the plot in Streamlit
st.pyplot(plt)

In [ ]:
print('The average person kills {:.4f} players on their own team'.format(df['TEAMKILLS'].mean()))
print('50% of people have killed ',df['TEAMKILLS'].quantile(0.50),' team players')
print('75% of people have killed ',df['TEAMKILLS'].quantile(0.75),' team players')
print('99% of people have killed ',df['TEAMKILLS'].quantile(0.99),' team players')
print('while the most kills recorded in the data is', df['TEAMKILLS'].max())

In [ ]:
# Step 1: Load the data from Snowflake into a Pandas DataFrame
df = session.table("gamestats").to_pandas()

# Step 2: Set up the plot size and create a jointplot
plt.figure(figsize=(15, 10))
sns.jointplot(x='WINPLACEPERC', y='TEAMKILLS', data=df, color='b')

# Step 3: Add axis labels
plt.xlabel('Win Place Percentage', fontsize=15)
plt.ylabel('Team Kills', fontsize=13)

# Step 4: Display the plot in Streamlit
st.pyplot(plt)

In [ ]:
data = df[['WINPLACEPERC']].copy()
data['TOTALDISTANCE'] = df['WALKDISTANCE'] + df['RIDEDISTANCE'] + df['SWIMDISTANCE']

# Summary statistics for the total distance travelled
print('The average person travelled {:.2f} m'.format(data['TOTALDISTANCE'].mean()))
print('25% of people have travelled {:.2f} m or less'.format(data['TOTALDISTANCE'].quantile(0.25)))
print('50% of people have travelled {:.2f} m or less'.format(data['TOTALDISTANCE'].quantile(0.50)))
print('75% of people have travelled {:.2f} m or less'.format(data['TOTALDISTANCE'].quantile(0.75)))
print('99% of people have travelled {:.2f} m or less'.format(data['TOTALDISTANCE'].quantile(0.99)))
print('The longest distance travelled in the data is {:.2f} m'.format(data['TOTALDISTANCE'].max()))


In [ ]:
data = df[['WINPLACEPERC']].copy()
data['TOTALDISTANCE'] = df['WALKDISTANCE'] + df['RIDEDISTANCE'] + df['SWIMDISTANCE']


# Step 2: Set up the plot size and create a jointplot
plt.figure(figsize=(15, 10))
sns.jointplot(x='WINPLACEPERC', y='TOTALDISTANCE', data=data, color='b')

# Step 3: Add axis labels
plt.xlabel('Win Place Percentage', fontsize=15)
plt.ylabel('Team Kills', fontsize=13)

# Step 4: Display the plot in Streamlit
st.pyplot(plt)

In [ ]:
# Summary statistics for the number of healing items used
print('The average person uses {:.2f} healing items'.format(df['HEALS'].mean()))
print('50% of people used {:.2f} healing items'.format(df['HEALS'].quantile(0.50)))
print('75% of people used {:.2f} healing items or less'.format(df['HEALS'].quantile(0.75)))
print('99% of people used {:.2f} healing items or less'.format(df['HEALS'].quantile(0.99)))
print('The doctor of the data used {:.2f} healing items'.format(df['HEALS'].max()))

In [ ]:
# Summary statistics for the number of boosting items used
print('The average person uses {:.2f} boosting items'.format(df['BOOSTS'].mean()))
print('50% of people used {:.2f} boosting items'.format(df['BOOSTS'].quantile(0.50)))
print('75% of people used {:.2f} boosting items or less'.format(df['BOOSTS'].quantile(0.75)))
print('99% of people used {:.2f} boosting items or less'.format(df['BOOSTS'].quantile(0.99)))
print('The addict of the data used {:.2f} boosting items'.format(df['BOOSTS'].max()))

In [ ]:
# Assume df is your DataFrame
data = df.copy()

# Filter out the top 1% to remove outliers
data = data[data['HEALS'] < data['HEALS'].quantile(0.99)]
data = data[data['BOOSTS'] < data['BOOSTS'].quantile(0.99)]

# Drop missing values
data = data.dropna(subset=['HEALS', 'BOOSTS', 'WINPLACEPERC'])

# Check if the DataFrame is empty
if data.empty:
    st.write("No data to display after filtering.")
else:
    # Create the figure and axes
    f, ax1 = plt.subplots(figsize=(20, 10))

    # Plot the data
    sns.pointplot(x='HEALS', y='WINPLACEPERC', data=data, color='red', alpha=1.0, ax=ax1)
    sns.pointplot(x='BOOSTS', y='WINPLACEPERC', data=data, color='blue', alpha=0.8, ax=ax1)

    # Add text labels
    plt.text(4, 0.6, 'HEALS', color='red', fontsize=17, style='italic')
    plt.text(4, 0.55, 'BOOSTS', color='blue', fontsize=17, style='italic')

    # Set labels and title
    plt.xlabel('Number of heal/boost items', fontsize=15, color='blue')
    plt.ylabel('Win Percentage', fontsize=15, color='blue')
    plt.title('HEALS vs BOOSTS', fontsize=20, color='blue')

    # Show grid
    plt.grid()

    # Display the plot in Streamlit
    st.pyplot(f)

In [ ]:
# Select only numeric columns for correlation
numeric_df = df.select_dtypes(include=[np.number])
    
# Compute the correlation matrix
corr_matrix = numeric_df.corr()
    
# Display the correlation matrix
st.subheader('Correlation Matrix')
st.dataframe(corr_matrix)
    
# Plot the heatmap
st.subheader('Correlation Heatmap')
fig, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(corr_matrix, annot=True, linewidths=.5, fmt='.2f', ax=ax)
    
# Display the matplotlib figure in Streamlit
st.pyplot(fig)


# Step 7: Clear the figure to avoid duplication
plt.clf()
plt.close()